# Search History Information Extraction & Analysis

## Project Overview
This notebook analyzes Google search history data to extract meaningful insights about user interests, preferences, and behavior patterns. Using Named Entity Recognition (NER) with OpenAI's GPT-4o-mini model and web scraping, we extract structured information from both search queries and visited pages to build a comprehensive user interest profile.

### Key Capabilities:
- Extract and categorize search queries by semantic meaning
- Analyze temporal patterns and search behavior
- Scrape and extract content from visited web pages
- Identify search clusters and user intent patterns
- Generate comprehensive user interest profiles
- Track API costs and optimize caching for efficiency

## Section 1: Load and Explore Search History Data

**Objective:** Load the Google search history JSON file and understand its structure, including all available fields and how records are organized.

**What this section does:**
- Loads search history from the JSON file
- Examines the data structure and available fields
- Shows sample records to understand the format
- Displays statistics about total records

In [ ]:
%pip install -q -r requirements.txt
import warnings
warnings.filterwarnings('ignore')

In [17]:
import json
import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter, defaultdict
import re
from urllib.parse import urlparse, parse_qs, unquote
import warnings
warnings.filterwarnings('ignore')

# Load the search history
with open('search_history.json', 'r') as f:
    search_data = json.load(f)

print(f"Total search history records: {len(search_data)}")
print(f"\nFirst 3 records structure:")
for i, record in enumerate(search_data[:3]):
    print(f"\n--- Record {i+1} ---")
    print(f"Keys: {record.keys()}")
    print(f"Title: {record.get('title', 'N/A')[:80]}")
    print(f"Time: {record.get('time', 'N/A')}")

Total search history records: 55383

First 3 records structure:

--- Record 1 ---
Keys: dict_keys(['header', 'title', 'titleUrl', 'time', 'products', 'activityControls'])
Title: Visited https://www.businessinsider.com/shivon-zilis-reported-mother-elon-musk-t
Time: 2024-06-23T22:21:50.431Z

--- Record 2 ---
Keys: dict_keys(['header', 'title', 'titleUrl', 'time', 'products', 'activityControls'])
Title: Visited Elon Musk and Shivon Zilis privately welcome third baby – NBC10 ...
Time: 2024-06-23T22:20:53.934Z

--- Record 3 ---
Keys: dict_keys(['header', 'title', 'titleUrl', 'time', 'products', 'activityControls', 'locationInfos'])
Title: Searched for elon musk shivon zilis
Time: 2024-06-23T22:20:47.560Z


## Section 2: Parse and Clean Search History Records

**Objective:** Process raw search history records to extract, classify, and structure the data for downstream analysis.

**What this section does:**
- Extracts search queries from title field using regex patterns
- Classifies activities (searches, page visits, notifications)
- Handles three types of records: "Searched for", "Visited", and other activities
- Converts timestamps to datetime format
- Adds temporal features (date, hour, day_of_week)
- Displays distribution of activity types and date range

In [27]:
# Function to extract search query from title
def extract_search_query(title):
    """
    Extract actual search query from the title field.
    Handles three types of records:
    1. "Searched for [query]" - explicit search queries
    2. "Visited [URL/Title]" - visited pages (not actual searches)
    3. Notifications and other non-search activities
    """
    if not title:
        return None
    
    # Pattern 1: "Searched for [query]"
    search_match = re.search(r'^Searched for (.+?)(?:\s*$|[\?&])', title)
    if search_match:
        return search_match.group(1).strip()
    
    # Pattern 2: Extract from notification topics
    if "notification" in title.lower() and "Including topics:" in title:
        return None  # Notifications aren't searches
    
    # Pattern 3: Visited URLs - extract from title content but mark differently
    if title.startswith("Visited"):
        return None
    
    return None

# Function to classify activity type
def classify_activity(record):
    """Classify the type of activity"""
    title = record.get('title', '')
    
    if title.startswith('Searched for'):
        return 'search_query'
    elif title.startswith('Visited'):
        return 'page_visit'
    elif 'notification' in title.lower():
        return 'notification'
    else:
        return 'other'

# Clean and structure the data
cleaned_records = []
for record in search_data:
    cleaned = {
        'timestamp': record.get('time'),
        'title': record.get('title'),
        'titleUrl': record.get('titleUrl'),
        'activity_type': classify_activity(record),
        'search_query': extract_search_query(record.get('title')),
        'location': record.get('locationInfos', [{}])[0].get('name') if record.get('locationInfos') else None,
    }
    cleaned_records.append(cleaned)

# Convert to DataFrame
df = pd.DataFrame(cleaned_records)

# Convert timestamp to datetime - use format='ISO8601' to handle mixed formats
df['timestamp'] = pd.to_datetime(df['timestamp'], format='ISO8601', utc=True)
df['date'] = df['timestamp'].dt.date
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.day_name()

# Remove duplicate searches and page visits (keep first occurrence)
before_rows = len(df)
search_dedup = (
    df[df['activity_type'] == 'search_query']
    .dropna(subset=['search_query'])
    .drop_duplicates(subset=['search_query'])
)
page_dedup = (
    df[df['activity_type'] == 'page_visit']
    .dropna(subset=['titleUrl'])
    .drop_duplicates(subset=['titleUrl'])
)
others = df[~df['activity_type'].isin(['search_query', 'page_visit'])]
df = pd.concat([search_dedup, page_dedup, others], ignore_index=True)
df = df.sort_values('timestamp').reset_index(drop=True)
after_rows = len(df)
duplicates_removed = before_rows - after_rows
print(f"Removed {duplicates_removed} duplicate search/page_visit records (kept first occurrences)")

# Show data summary
print(f"Total records: {len(df)}")
print(f"\nActivity type distribution:")
print(df['activity_type'].value_counts())
print(f"\nDate range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"\nSample cleaned records:")
print(df[['activity_type', 'search_query', 'title','titleUrl','timestamp',]].head(10))

Removed 7405 duplicate search/page_visit records (kept first occurrences)
Total records: 47978

Activity type distribution:
activity_type
search_query    25807
page_visit      19826
other            2187
notification      158
Name: count, dtype: int64

Date range: 2017-06-08 16:45:50.139000+00:00 to 2024-06-23 22:21:50.431000+00:00

Sample cleaned records:
  activity_type                                 search_query  \
0  search_query  investment banking networking events london   
1    page_visit                                         None   
2  search_query          blackstone's women networking event   
3  search_query                                   blackstone   
4    page_visit                                         None   
5  search_query                                         face   
6    page_visit                                         None   
7  search_query      how to find people's emails on data.com   
8    page_visit                                         None   
9

In [19]:
#run twice to verify if downloaded
!python /Users/shreyasjagannath/dev/fabric/onfabric-data-science-interview/download_nltk_data.py

NLTK data directory: /Users/shreyasjagannath/nltk_data
→ Downloading punkt (tokenizers)… [nltk_data] Downloading package punkt to
[nltk_data]     /Users/shreyasjagannath/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
✓
→ Downloading punkt_tab (tokenizers)… [nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/shreyasjagannath/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
✓
→ Downloading stopwords (corpora)… [nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shreyasjagannath/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
✓
→ Downloading wordnet (corpora)… [nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shreyasjagannath/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
⚠ not found after download
→ Downloading averaged_perceptron_tagger (taggers)… [nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/shreyasjagannath/nltk_data...
[nltk

In [28]:
import re
from difflib import SequenceMatcher
from datetime import timedelta
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings('ignore')

# Initialize NLTK tools
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Additional domain-specific stopwords to exclude
domain_stopwords = {
    'visited', 'searched', 'www', 'http', 'https', 'com', 'org', 'net', 'co', 'uk', 'us',
    'page', 'site', 'link', 'url', 'web', 'search', 'result', 'article', 'post', 'blog',
    'click', 'view', 'visit', 'read', 'news'
}
stop_words.update(domain_stopwords)

def extract_keywords_nltk(text):
    """
    Extract meaningful keywords using NLTK NLP techniques.
    Applies: Tokenization -> Lowercase -> Stopword Removal -> Lemmatization
    Returns a set of normalized keywords
    """
    if not text or not isinstance(text, str):
        return set()
    
    try:
        # Step 1: Tokenization - Split text into words
        tokens = word_tokenize(text.lower())
        
        # Step 2: Remove special characters and filter alphanumeric tokens
        tokens = [token for token in tokens if token.isalnum()]
        
        # Step 3: Stopword Removal - Remove common words
        tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
        
        # Step 4: Lemmatization - Convert words to base form
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
        
        # Convert to set to get unique keywords
        keywords = set(lemmatized_tokens)
        
        return keywords
    
    except Exception as e:
        print(f"Error processing text: {e}")
        return set()

def extract_keywords_lemmatized(text):
    """
    Extract keywords using only lemmatization (identical to extract_keywords_nltk now)
    Useful for preserving word meaning
    """
    if not text or not isinstance(text, str):
        return set()
    
    try:
        tokens = word_tokenize(text.lower())
        tokens = [token for token in tokens if token.isalnum()]
        tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
        return set(lemmatized_tokens)
    except Exception as e:
        return set()

def calculate_keyword_similarity(keywords1, keywords2):
    """
    Calculate similarity between two sets of keywords using Jaccard similarity.
    Returns a score between 0 and 1.
    
    Jaccard = (intersection size) / (union size)
    """
    if not keywords1 or not keywords2:
        return 0.0
    
    intersection = len(keywords1 & keywords2)
    union = len(keywords1 | keywords2)
    
    if union == 0:
        return 0.0
    
    return intersection / union

def extract_search_clusters(df, time_window_minutes=10, keyword_similarity_threshold=0.3, 
                             max_lookahead=10, min_cluster_size=2, use_stemming=False):
    """
    Extract search clusters from the dataframe based on keyword similarity and timestamps.
    Uses NLTK for advanced text processing.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Input dataframe with columns: timestamp, title, search_query, activity_type
    
    time_window_minutes : int, default=10
        Maximum time window (in minutes) for entries to be considered in the same cluster
    
    keyword_similarity_threshold : float, default=0.3
        Minimum keyword similarity (0-1) to consider entries as related
    
    max_lookahead : int, default=10
        Only compare each entry with up to N entries after it (performance optimization)
    
    min_cluster_size : int, default=2
        Minimum number of entries required to form a cluster
    
    use_stemming : bool, default=True (ignored, always lemmatizes now)
    
    Returns:
    --------
    tuple : (clusters_df, cluster_info)
        - clusters_df: DataFrame with all entries that are part of clusters, with cluster_id
        - cluster_info: Dictionary with summary statistics about detected clusters
    
    Example:
    --------
    clusters_df, info = extract_search_clusters(df)
    print(f"Found {info['num_clusters']} clusters")
    print(clusters_df.head())
    """
    
    # Sort by timestamp for sequential processing
    sorted_df = df.sort_values('timestamp').reset_index(drop=True)
    
    # Always use lemmatization now
    keyword_extractor = extract_keywords_nltk
    
    # Dictionary to store cluster assignments: index -> cluster_id
    cluster_assignments = {}
    cluster_counter = 0
    
    # Keep track of which indices have been assigned to clusters
    assigned_indices = set()
    
    # Pre-compute keywords for all entries to improve performance
    print("Extracting keywords from all entries...")
    entry_keywords = {}
    for i in range(len(sorted_df)):
        current_row = sorted_df.iloc[i]
        text = str(current_row['search_query']) if pd.notna(current_row['search_query']) else current_row['title']
        entry_keywords[i] = keyword_extractor(text)
    
    print(f"✓ Keyword extraction complete for {len(entry_keywords)} entries\n")
    
    # Process each entry
    for i in range(len(sorted_df)):
        # Skip if already assigned to a cluster
        if i in assigned_indices:
            continue
        
        current_row = sorted_df.iloc[i]
        current_timestamp = current_row['timestamp']
        current_keywords = entry_keywords[i]
        
        # If no keywords extracted, skip
        if not current_keywords:
            continue
        
        # Find related entries within lookahead window
        cluster_indices = [i]  # Start with current entry
        
        # Look ahead at most max_lookahead entries
        lookahead_end = min(i + 1 + max_lookahead, len(sorted_df))
        
        for j in range(i + 1, lookahead_end):
            if j in assigned_indices:
                continue
            
            next_row = sorted_df.iloc[j]
            next_timestamp = next_row['timestamp']
            
            # Check time window constraint
            time_diff = (next_timestamp - current_timestamp).total_seconds() / 60
            if time_diff > time_window_minutes:
                break  # No point checking further entries (sorted by time)
            
            # Get pre-computed keywords
            next_keywords = entry_keywords[j]
            
            if not next_keywords:
                continue
            
            # Calculate keyword similarity
            similarity = calculate_keyword_similarity(current_keywords, next_keywords)
            
            # If similar enough, add to cluster
            if similarity >= keyword_similarity_threshold:
                cluster_indices.append(j)
        
        # Only create cluster if it has minimum size
        if len(cluster_indices) >= min_cluster_size:
            for idx in cluster_indices:
                cluster_assignments[idx] = cluster_counter
                assigned_indices.add(idx)
            cluster_counter += 1
    
    # Create result dataframe with cluster assignments
    if cluster_assignments:
        cluster_df = sorted_df.loc[list(cluster_assignments.keys())].copy()
        cluster_df['cluster_id'] = cluster_df.index.map(cluster_assignments)
        
        # Sort by cluster_id and timestamp for better organization
        cluster_df = cluster_df.sort_values(['cluster_id', 'timestamp']).reset_index(drop=True)
    else:
        # No clusters found
        cluster_df = pd.DataFrame(columns=sorted_df.columns.tolist() + ['cluster_id'])
    
    # Generate cluster information statistics
    cluster_info = {
        'num_clusters': cluster_counter,
        'total_clustered_entries': len(assigned_indices),
        'total_entries': len(sorted_df),
        'clustering_percentage': (len(assigned_indices) / len(sorted_df) * 100) if len(sorted_df) > 0 else 0,
        'time_window_minutes': time_window_minutes,
        'similarity_threshold': keyword_similarity_threshold,
        'max_lookahead': max_lookahead,
        'min_cluster_size': min_cluster_size,
        'nlp_method': 'Lemmatization (WordNet)',
    }
    
    # Add per-cluster statistics
    if cluster_counter > 0:
        cluster_sizes = {}
        for idx, cid in cluster_assignments.items():
            cluster_sizes[cid] = cluster_sizes.get(cid, 0) + 1
        
        cluster_info['cluster_sizes'] = cluster_sizes
        cluster_info['avg_cluster_size'] = sum(cluster_sizes.values()) / len(cluster_sizes)
        cluster_info['largest_cluster_size'] = max(cluster_sizes.values())
    
    return cluster_df, cluster_info


print("=" * 100)
print("SEARCH CLUSTER EXTRACTION WITH NLTK")
print("=" * 100)
print(f"\nNLP Techniques Enabled:")
print(f"  ✓ Tokenization - Split text into individual words/tokens")
print(f"  ✓ Stopword Removal - Remove common English words")
print(f"  ✓ Lemmatization - Convert words to base/dictionary form (e.g., 'running' → 'run')")
print(f"\nInput dataframe shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nFunction parameters:")
print(f"  - time_window_minutes: 10 (entries within 10 min window)")
print(f"  - keyword_similarity_threshold: 0.3 (30% keyword overlap)")
print(f"  - max_lookahead: 10 (compare with up to 10 following entries)")
print(f"  - min_cluster_size: 2 (minimum 2 entries per cluster)")
print(f"  - use_stemming: False (stemming removed, always lemmatizes)")
print(f"\n✅ NLTK tools initialized successfully!")
print(f"Ready to call: clusters_df, info = extract_search_clusters(df)")


SEARCH CLUSTER EXTRACTION WITH NLTK

NLP Techniques Enabled:
  ✓ Tokenization - Split text into individual words/tokens
  ✓ Stopword Removal - Remove common English words
  ✓ Lemmatization - Convert words to base/dictionary form (e.g., 'running' → 'run')

Input dataframe shape: (47978, 9)
Columns: ['timestamp', 'title', 'titleUrl', 'activity_type', 'search_query', 'location', 'date', 'hour', 'day_of_week']

Function parameters:
  - time_window_minutes: 10 (entries within 10 min window)
  - keyword_similarity_threshold: 0.3 (30% keyword overlap)
  - max_lookahead: 10 (compare with up to 10 following entries)
  - min_cluster_size: 2 (minimum 2 entries per cluster)
  - use_stemming: False (stemming removed, always lemmatizes)

✅ NLTK tools initialized successfully!
Ready to call: clusters_df, info = extract_search_clusters(df)


In [29]:
# Run the search cluster extraction with NLTK
print("=" * 100)
print("EXTRACTING SEARCH CLUSTERS FROM DATA (NLTK-POWERED)")
print("=" * 100)

clusters_df, cluster_info = extract_search_clusters(
    df,
    time_window_minutes=20,           # 20-minute time window
    keyword_similarity_threshold=0.3,  # 30% keyword overlap
    max_lookahead=20,                  # Compare with 20 entries ahead
    min_cluster_size=1,                # Minimum 1 entry per cluster
    use_stemming=False                 # Stemming removed, always lemmatizes
 )

print(f"\n✓ Clustering complete!")
print(f"\nCluster Statistics:")
print(f"  Total clusters found: {cluster_info['num_clusters']}")
print(f"  Total clustered entries: {cluster_info['total_clustered_entries']} / {cluster_info['total_entries']}")
print(f"  Clustering percentage: {cluster_info['clustering_percentage']:.1f}%")
print(f"  Average cluster size: {cluster_info.get('avg_cluster_size', 0):.1f}")
print(f"  Largest cluster size: {cluster_info.get('largest_cluster_size', 0)}")
print(f"  NLP Method: {cluster_info['nlp_method']}")

# Display cluster distribution
if cluster_info['num_clusters'] > 0:
    print(f"\n📊 Cluster Size Distribution:")
    size_counts = {}
    for size in cluster_info['cluster_sizes'].values():
        size_counts[size] = size_counts.get(size, 0) + 1
    
    for size in sorted(size_counts.keys()):
        count = size_counts[size]
        print(f"  {size} entries: {count} clusters")

print(f"\n✓ Result dataframe shape: {clusters_df.shape}")
print(f"  Columns: {clusters_df.columns.tolist()}")


EXTRACTING SEARCH CLUSTERS FROM DATA (NLTK-POWERED)
Extracting keywords from all entries...
Extracting keywords from all entries...
✓ Keyword extraction complete for 47978 entries

✓ Keyword extraction complete for 47978 entries


✓ Clustering complete!

Cluster Statistics:
  Total clusters found: 28566
  Total clustered entries: 38649 / 47978
  Clustering percentage: 80.6%
  Average cluster size: 1.4
  Largest cluster size: 13
  NLP Method: Lemmatization (WordNet)

📊 Cluster Size Distribution:
  1 entries: 21325 clusters
  2 entries: 5340 clusters
  3 entries: 1328 clusters
  4 entries: 369 clusters
  5 entries: 117 clusters
  6 entries: 48 clusters
  7 entries: 22 clusters
  8 entries: 9 clusters
  9 entries: 3 clusters
  10 entries: 2 clusters
  12 entries: 1 clusters
  13 entries: 2 clusters

✓ Result dataframe shape: (38649, 10)
  Columns: ['timestamp', 'title', 'titleUrl', 'activity_type', 'search_query', 'location', 'date', 'hour', 'day_of_week', 'cluster_id']

✓ Clustering comp

In [30]:
# Demonstrate NLTK preprocessing techniques
print("=" * 100)
print("NLTK TEXT PREPROCESSING DEMONSTRATION")
print("=" * 100)

test_texts = [
    "Hermes sandals for women designer luxury",
    "searching for luxury hermes shoe designs",
    "visited luxury designer sandals online",
    "Real estate property in Bangalore HSR",
    "Properties for sale in HSR layout locations",
]

print("\nText Processing Pipeline:\n")
print(f"{'Original Text':<50} → {'Tokens':<50}")
print("-" * 100)

for text in test_texts[:3]:
    tokens = word_tokenize(text.lower())
    print(f"{text:<50} → {tokens}")

print("\n" + "-" * 100)
print(f"{'After Tokenization + Stopword Removal':<50} → {'After Lemmatization':<50}")
print("-" * 100)

for text in test_texts[:3]:
    # Step-by-step processing
    tokens = word_tokenize(text.lower())
    filtered_tokens = [t for t in tokens if t.isalnum() and t not in stop_words and len(t) > 2]
    lemmatized = [lemmatizer.lemmatize(t) for t in filtered_tokens]
    
    print(f"{str(filtered_tokens):<50} → {str(lemmatized)[:49]}")

print("\n" + "-" * 100)
print(f"{'Lemmatized':<50} → {'Final Keywords':<50}")
print("-" * 100)

for text in test_texts[:3]:
    tokens = word_tokenize(text.lower())
    filtered_tokens = [t for t in tokens if t.isalnum() and t not in stop_words and len(t) > 2]
    lemmatized = [lemmatizer.lemmatize(t) for t in filtered_tokens]
    keywords = set(lemmatized)
    
    print(f"{str(lemmatized):<50} → {str(sorted(keywords))[:49]}")

print("\n" + "=" * 100)
print("FINAL EXTRACTED KEYWORDS (Using Lemmatization Only):")
print("=" * 100)

for i, text in enumerate(test_texts, 1):
    keywords = extract_keywords_nltk(text)
    print(f"\n{i}. Original: '{text}'")
    print(f"   Keywords: {sorted(keywords)}")

# Demonstrate similarity calculation
print("\n\n" + "=" * 100)
print("KEYWORD SIMILARITY ANALYSIS:")
print("=" * 100)

text1_keywords = extract_keywords_nltk(test_texts[0])
text2_keywords = extract_keywords_nltk(test_texts[1])
text3_keywords = extract_keywords_nltk(test_texts[2])

similarity_1_2 = calculate_keyword_similarity(text1_keywords, text2_keywords)
similarity_1_3 = calculate_keyword_similarity(text1_keywords, text3_keywords)
similarity_2_3 = calculate_keyword_similarity(text2_keywords, text3_keywords)

print(f"\nText 1: '{test_texts[0]}'")
print(f"  Keywords: {sorted(text1_keywords)}\n")

print(f"Text 2: '{test_texts[1]}'")
print(f"  Keywords: {sorted(text2_keywords)}")
print(f"  Similarity with Text 1: {similarity_1_2:.2%}\n")

print(f"Text 3: '{test_texts[2]}'")
print(f"  Keywords: {sorted(text3_keywords)}")
print(f"  Similarity with Text 1: {similarity_1_3:.2%}")
print(f"  Similarity with Text 2: {similarity_2_3:.2%}\n")

print(f"Interpretation:")
print(f"  • Texts with similar keywords (>30%) will be clustered together")
print(f"  • Related searches about 'hermes' and 'sandals' will form clusters")
print(f"  • Real estate searches form separate clusters")


NLTK TEXT PREPROCESSING DEMONSTRATION

Text Processing Pipeline:

Original Text                                      → Tokens                                            
----------------------------------------------------------------------------------------------------
Hermes sandals for women designer luxury           → ['hermes', 'sandals', 'for', 'women', 'designer', 'luxury']
searching for luxury hermes shoe designs           → ['searching', 'for', 'luxury', 'hermes', 'shoe', 'designs']
visited luxury designer sandals online             → ['visited', 'luxury', 'designer', 'sandals', 'online']

----------------------------------------------------------------------------------------------------
After Tokenization + Stopword Removal              → After Lemmatization                               
----------------------------------------------------------------------------------------------------
['hermes', 'sandals', 'women', 'designer', 'luxury'] → ['hermes', 'sandal', 'woman', 'de

In [31]:
# Display example clusters with detailed information and NLTK-extracted keywords
print("\n" + "=" * 100)
print("EXAMPLE SEARCH CLUSTERS - DETAILED VIEW (WITH NLTK ANALYSIS)")
print("=" * 100)

if len(clusters_df) > 0:
    unique_clusters = clusters_df['cluster_id'].unique()
    num_to_show = min(5, len(unique_clusters))
    
    for cluster_num, cluster_id in enumerate(unique_clusters[:num_to_show], 1):
        cluster_entries = clusters_df[clusters_df['cluster_id'] == cluster_id].copy()
        
        print(f"\n{'█' * 100}")
        print(f"CLUSTER {int(cluster_id) + 1} (Size: {len(cluster_entries)} entries)")
        print(f"{'█' * 100}")
        
        # Extract NLTK keywords for the cluster
        lemma_keywords = set()
        
        for _, row in cluster_entries.iterrows():
            text = str(row['search_query']) if pd.notna(row['search_query']) else row['title']
            lemma_kw = extract_keywords_nltk(text)
            lemma_keywords.update(lemma_kw)
        
        print(f"\n🏷️  NLTK-Extracted Keywords:")
        print(f"     Lemmatized:     {', '.join(sorted(lemma_keywords)[:15])}")
        
        # Time span
        time_span = (cluster_entries['timestamp'].max() - cluster_entries['timestamp'].min()).total_seconds() / 60
        print(f"\n⏱️  Time Span: {time_span:.1f} minutes")
        print(f"📅 Activities: {cluster_entries['activity_type'].value_counts().to_dict()}")
        
        print(f"\n📝 Entries in cluster:")
        for idx, (_, row) in enumerate(cluster_entries.iterrows(), 1):
            timestamp_str = row['timestamp'].strftime('%Y-%m-%d %H:%M:%S')
            activity = row['activity_type']
            title = row['title'][:70] if len(row['title']) > 70 else row['title']
            
            print(f"\n   {idx}. [{timestamp_str}] {activity.upper()}")
            print(f"      Title: {title}")
            
            if pd.notna(row['search_query']):
                keywords = extract_keywords_nltk(row['search_query'])
                print(f"      Query: {row['search_query']}")
                print(f"      Keywords: {', '.join(sorted(keywords))}")
else:
    print("\n⚠️  No clusters found with current parameters.")
    print("Try adjusting: keyword_similarity_threshold (lower), time_window_minutes (higher), min_cluster_size (lower)")



EXAMPLE SEARCH CLUSTERS - DETAILED VIEW (WITH NLTK ANALYSIS)

████████████████████████████████████████████████████████████████████████████████████████████████████
CLUSTER 1 (Size: 1 entries)
████████████████████████████████████████████████████████████████████████████████████████████████████

🏷️  NLTK-Extracted Keywords:
     Lemmatized:     banking, event, investment, london, networking

⏱️  Time Span: 0.0 minutes
📅 Activities: {'search_query': 1}

📝 Entries in cluster:

   1. [2017-06-08 16:45:50] SEARCH_QUERY
      Title: Searched for investment banking networking events london
      Query: investment banking networking events london
      Keywords: banking, event, investment, london, networking

████████████████████████████████████████████████████████████████████████████████████████████████████
CLUSTER 2 (Size: 1 entries)
████████████████████████████████████████████████████████████████████████████████████████████████████

🏷️  NLTK-Extracted Keywords:
     Lemmatized:     blackstone

## Section 3: Advanced NER-Based Entity Extraction Using OpenAI

**Objective:** Use OpenAI's GPT-4o-mini model to perform sophisticated Named Entity Recognition and extract structured information (category, topic, entities) from search queries.

**What this section does:**
- Initializes OpenAI client with API credentials
- Defines extraction prompt for structured information extraction
- Implements extraction function that returns category, topic, and entities
- Tests extraction on sample queries
- Sets up caching system to avoid re-processing identical texts

In [33]:
# Install and configure OpenAI
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables from .env (if present)
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not found. Set it in your environment or .env file.")

# Initialize OpenAI client
client = OpenAI(api_key=api_key)

print("OpenAI client initialized successfully!")
print("Using model: gpt-5-nano via responses API for NER extraction")

OpenAI client initialized successfully!
Using model: gpt-5-nano via responses API for NER extraction


In [ ]:
# Define the extraction prompt for OpenAI
EXTRACTION_SYSTEM_PROMPT = """You are an expert at analyzing search queries and web page titles to extract structured information.

For each input text, extract:
1. CATEGORY: The main semantic category (choose from: Fashion & Accessories, Real Estate & Property, Technology & Innovation, Wellness & Health, Travel & Transportation, Business & Finance, News & Media, Shopping & Retail, Entertainment, Books & Learning, General Interest)
2. TOPIC: The main subject or focus of the query/title
3. ITEMS: Specific entities mentioned (brands, products, people, places, organizations)

Examples:
- "Hermes sandals for women" → Category: Fashion & Accessories, Topic: Sandals, Items: ["Hermes"]
- "Real estate property in HSR Layout Bangalore" → Category: Real Estate & Property, Topic: Property search, Items: ["HSR Layout", "Bangalore"]
- "How to bake sourdough bread" → Category: Books & Learning, Topic: Baking, Items: ["sourdough"]
- "children using smartphones too often" → Category: Wellness & Health, Topic: Smartphone usage, Items: ["children", "smartphones"]

Return ONLY valid JSON in this exact format:
{
  "category": "Category Name",
  "topic": "Main topic",
  "items": ["item1", "item2"]
}"""

import re

def extract_entities_and_context(text):
    """
    Extract named entities and categorize them using OpenAI gpt-5-nano via the Responses API.

    Returns:
    {
        'category': str,
        'topic': str,
        'items': list,
        'raw_text': str
    }
    """
    if not text or not isinstance(text, str):
        return {
            'category': 'Uncategorized',
            'topic': 'Unknown',
            'items': [],
            'raw_text': text
        }
    
    def try_parse_json(json_str):
        try:
            return json.loads(json_str)
        except Exception as e:
            # Try to repair by truncating at the first unmatched closing brace
            last_brace = json_str.rfind('}')
            if last_brace != -1:
                try:
                    return json.loads(json_str[:last_brace+1])
                except Exception as e2:
                    pass
            return None
    
    try:
        # Call OpenAI Responses API with gpt-5-nano (response_format not supported in this SDK build)
        response = client.responses.create(
            model="gpt-5-nano",
            instructions=EXTRACTION_SYSTEM_PROMPT + "\nReturn JSON with keys: category, topic, items.",
            input=f"Extract structured information from: {text}",
            reasoning={"effort": "low"},
        )
        
        # Use regex to extract the first JSON object from the response text
        response_text = response.output_text
        json_match = re.search(r'\{[\s\S]*?\}', response_text)
        result = None
        if json_match:
            json_str = json_match.group(0)
            result = try_parse_json(json_str)
            if result is None:
                print(f"JSON decode error (regex-extracted, repaired): Could not parse after repair.")
        else:
            # Fallback: try to parse the whole response as JSON (legacy)
            result = try_parse_json(response_text)
            if result is None:
                print(f"JSON decode error (full text, repaired): Could not parse after repair.")
        
        if result is None:
            return {
                'category': 'General Interest',
                'topic': text[:30] if len(text) > 30 else text,
                'items': [],
                'raw_text': text
            }
        
        # If result is a list, take the first dict
        if isinstance(result, list):
            if len(result) > 0 and isinstance(result[0], dict):
                result = result[0]
            else:
                return {
                    'category': 'General Interest',
                    'topic': text[:30] if len(text) > 30 else text,
                    'items': [],
                    'raw_text': text
                }
        
        return {
            'category': result.get('category', 'General Interest'),
            'topic': result.get('topic', 'Not specified'),
            'items': result.get('items', []),
            'raw_text': text
        }
    
    except Exception as e:
        # Fallback to simple keyword matching if API fails
        print(f"API error for '{text[:50]}...': {e}")
        return {
            'category': 'General Interest',
            'topic': text[:30] if len(text) > 30 else text,
            'items': [],
            'raw_text': text
        }

# Test the extraction function
print("=== TESTING OPENAI NER EXTRACTION ===\n")
test_queries = [
    "Hermes sandals for women",
    "How to bake sourdough bread",
    "Real estate property in HSR Layout Bangalore",
    "Children smartphone addiction research"
 ]

for query in test_queries:
    result = extract_entities_and_context(query)
    print(f"Query: {query}")
    print(f"  Category: {result['category']}")
    print(f"  Topic: {result['topic']}")
    print(f"  Items: {result['items']}")
    print()

=== TESTING OPENAI NER EXTRACTION ===

Query: Hermes sandals for women
  Category: Fashion & Accessories
  Topic: Sandals
  Items: ['Hermes']

Query: Hermes sandals for women
  Category: Fashion & Accessories
  Topic: Sandals
  Items: ['Hermes']

Query: How to bake sourdough bread
  Category: Books & Learning
  Topic: Baking
  Items: ['sourdough']

Query: How to bake sourdough bread
  Category: Books & Learning
  Topic: Baking
  Items: ['sourdough']

Query: Real estate property in HSR Layout Bangalore
  Category: Real Estate & Property
  Topic: Property search
  Items: ['HSR Layout', 'Bangalore']

Query: Real estate property in HSR Layout Bangalore
  Category: Real Estate & Property
  Topic: Property search
  Items: ['HSR Layout', 'Bangalore']

Query: Children smartphone addiction research
  Category: Wellness & Health
  Topic: Smartphone addiction in children
  Items: ['children', 'smartphones']

Query: Children smartphone addiction research
  Category: Wellness & Health
  Topic: Smar

In [35]:
# Parallelized cluster topic/category/entity extraction with OpenAI NER and page scraping, with progress display
import concurrent.futures
import requests
import time
from urllib.parse import urlparse
from difflib import SequenceMatcher
from tqdm import tqdm

# Helper: check if URL is a PDF
def is_pdf_url(url):
    return url.lower().endswith('.pdf') or 'application/pdf' in url.lower()

# Helper: fetch and return main text content from a URL (skip PDFs)
def fetch_page_content(url, timeout=10):
    try:
        if is_pdf_url(url):
            return None, 'PDF skipped'
        headers = {'User-Agent': 'Mozilla/5.0'}
        resp = requests.get(url, headers=headers, timeout=timeout)
        ct = resp.headers.get('Content-Type', '')
        if 'pdf' in ct or is_pdf_url(url):
            return None, 'PDF skipped'
        if 'text/html' not in ct:
            return None, f'Non-HTML content: {ct}'
        # crude main text extraction
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(resp.text, 'html.parser')
        # Remove scripts/styles
        for tag in soup(['script', 'style', 'noscript']):
            tag.decompose()
        text = ' '.join(soup.stripped_strings)
        return text[:5000], None  # limit to 5k chars
    except Exception as e:
        return None, f'Error: {e}'

# Helper: similarity between query and url/title
def simple_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Main cluster summarization function
def summarize_cluster(cluster_df, openai_extractor, max_api_retries=3, api_delay=1.0):
    queries = cluster_df[cluster_df['activity_type'] == 'search_query']['search_query'].dropna().tolist()
    page_visits = cluster_df[cluster_df['activity_type'] == 'page_visit']['titleUrl'].dropna().tolist()
    queries_text = ' '.join(queries)

    # Collect content/status for every visited page in the cluster
    page_entries = []
    for url in page_visits:
        if not isinstance(url, str):
            continue
        content, status = fetch_page_content(url)
        page_entries.append({'url': url, 'content': content, 'status': status})

    # Compose context that includes all queries and all visited pages
    context_parts = []
    if queries:
        context_parts.append('Queries: ' + ' | '.join(queries))
    for entry in page_entries:
        url = entry['url']
        content = entry['content']
        status = entry['status']
        if content:
            context_parts.append(f'Website: {url}\nContent: {content}')
        else:
            context_parts.append(f'Website: {url}\nContent: {status or "(unavailable)"}')
    context = '\n'.join(context_parts)

    # OpenAI NER extraction with retry
    for attempt in range(max_api_retries):
        try:
            result = openai_extractor(context)
            break
        except Exception as e:
            if attempt < max_api_retries - 1:
                time.sleep(api_delay * (2 ** attempt))
            else:
                result = {'category': 'Unknown', 'topic': 'Unknown', 'items': [], 'raw_text': context, 'error': str(e)}
    return {
        'category': result.get('category', 'Unknown'),
        'topic': result.get('topic', 'Unknown'),
        'items': result.get('items', []),
        'raw_text': context
    }

# Wrapper for all clusters with progress bar
def summarize_all_clusters(clusters_df, openai_extractor, max_workers=3):
    cluster_results = {}
    if 'cluster_id' not in clusters_df.columns:
        print('No clusters to summarize.')
        return cluster_results
    grouped = list(clusters_df.groupby('cluster_id'))
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for cid, group in grouped:
            futures.append((cid, executor.submit(summarize_cluster, group, openai_extractor)))
        for cid, future in tqdm(futures, desc='Extracting clusters', unit='cluster'):
            try:
                cluster_results[cid] = future.result()
            except Exception as e:
                cluster_results[cid] = {'category': 'Unknown', 'topic': 'Unknown', 'items': [], 'raw_text': '', 'error': str(e)}
    return cluster_results

# Usage example (requires OpenAI extractor function):
# cluster_summaries = summarize_all_clusters(clusters_df, extract_entities_and_context, max_workers=3)
# print(cluster_summaries)

In [36]:
# Run cluster extraction and save results to JSON
import json

# Run extraction (this may take time and use OpenAI API credits)
cluster_summaries = summarize_all_clusters(clusters_df, extract_entities_and_context, max_workers=6)

# Save to JSON file
output_path = 'cluster_summaries.json'
with open(output_path, 'w') as f:
    json.dump(cluster_summaries, f, indent=2)

print(f"Cluster summaries saved to {output_path} ({len(cluster_summaries)} clusters)")

Extracting clusters:   2%|▏         | 574/28566 [04:24<7:58:28,  1.03s/cluster] 

JSON decode error (full text, repaired): Could not parse after repair.


Extracting clusters:   7%|▋         | 2131/28566 [15:51<2:11:30,  3.35cluster/s] 

JSON decode error (regex-extracted, repaired): Could not parse after repair.


Extracting clusters:  46%|████▌     | 13138/28566 [1:52:12<1:44:38,  2.46cluster/s] 

...

JSON decode error (regex-extracted, repaired): Could not parse after repair.


Extracting clusters:  64%|██████▍   | 18249/28566 [2:31:29<2:06:02,  1.36cluster/s]

JSON decode error (regex-extracted, repaired): Could not parse after repair.


Extracting clusters: 100%|██████████| 28566/28566 [4:05:58<00:00,  1.94cluster/s]   


Cluster summaries saved to cluster_summaries.json (28566 clusters)


## Section 4: Summary and Generated Files

**Objective:** Provide a summary of all output files generated by this analysis and usage instructions.

**What this section does:**
- Lists all generated files with descriptions and sizes
- Shows when files were created and updated
- Provides next steps and usage examples
- Explains how to load and use the structured output

In [38]:
# Display summary of all generated files
import os

print("=" * 80)
print("GENERATED FILES SUMMARY")
print("=" * 80)

files_to_check = [
    ('cluster_summaries.json', 'Complete analysis with categories, topics, entities for clusters')
]

print("\n📁 Output Files:\n")

for filename, description in files_to_check:
    if os.path.exists(filename):
        file_size = os.path.getsize(filename)
        file_size_mb = file_size / (1024 * 1024)
        
        print(f"✓ {filename}")
        print(f"  Description: {description}")
        print(f"  Size: {file_size_mb:.2f} MB")
        print()
    else:
        print(f"⚠ {filename} - Not found")
        print(f"  Description: {description}")
        print()

print("=" * 80)

GENERATED FILES SUMMARY

📁 Output Files:

✓ cluster_summaries.json
  Description: Complete analysis with categories, topics, entities for clusters
  Size: 41.30 MB

